# Install and import all the dependencies

In [ ]:
# Add path to the data directory, do not include the directory name
import sys
sys.path.append('/content/drive/MyDrive/HT')

# Fetch the relative path variables
from Data.constants import SDK_PATH, DATA_PATH_HIGH_LEVEL, DATA_PATH_LABELS, ALIGNED_DATA_PATH_HIGH_LEVEL, ALIGNED_DATA_PATH_LABELS, WORD_EMB_PATH, CACHE_PATH, DATA_PATH_RAW

from Models import TextEmbedder, VisualsEmbedder, AcousticsEmbedder

# Get the embeddings of all three modalities

In [ ]:
text_emb = TextEmbedder.get_embeddings()
visual_emb = VisualsEmbedder.get_embeddings()
acoustic_emb = AcousticsEmbedder.get_embeddings()

# Deep Canonical Correlation Analysis Model

In [10]:
! pip install scikit-learn

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.cross_decomposition import CCA

## Encoder

In [7]:
class Encoder(nn.Module):
    def __init__(self, in_dim, embed_dim):
        super(Encoder, self).__init__()

        # Text Modality
        self.linear1 = nn.Linear(in_dim, 150)
        self.linear2 = nn.Linear(150, 50)

        # Visual Modality
        self.norm = nn.BatchNorm1d(embed_dim)
        self.conv1 = nn.Conv1d(1, 16, 3, 1)
        self.conv2 = nn.Conv1d(16, 32, 3, 1)
        self.linear3 = nn.Linear(int(embed_dim/4), 50)

        # Acoustic Modality
        self.conv3 = nn.Conv1d(1, 16, 3, 1)
        self.conv4 = nn.Conv1d(16, 32, 3, 1)
        self.linear4 = nn.Linear(int(embed_dim/4), 50)

    def forward(self, t, v, a):

        # Text Modality
        t_ = nn.ReLU(self.linear1(t))
        t_ = nn.ReLU(self.linear2(t_))

        # Visual Modality
        v_ = nn.ReLU(self.norm(self.conv1(v)))
        v_ = nn.ReLU(self.norm(self.conv2(v_)))
        v_ = nn.ReLU(self.linear3(v_))

        # Acoustic Modality
        a_ = nn.ReLU(self.norm(self.conv3(a)))
        a_ = nn.ReLU(self.norm(self.conv4(a_)))
        a_ = nn.ReLU(self.linear4(a_))

        return torch.cat((t_, v_, a_), dim=1)

## Canonical Correlation Analysis

In [8]:
model = Encoder(300, 74)
inputs = Encoder.forward(text_emb, visual_emb, acoustic_emb)

cca_module = CCA(n_components=3)
outputs = cca_module.fit_transform(inputs)

# A Function to send the fused modalities

In [ ]:
def get_fusion():
    return outputs

# Cross Fusion with standard attention module

Credit for the base self-attention:
https://github.com/pbloem/former/blob/master/former/modules.py by *pbloem*

https://discuss.pytorch.org/t/my-implementation-of-self-attention/81130 by *omer_sahban*

https://pytorch.org/docs/master/_modules/torch/nn/modules/activation.html#MultiheadAttention


In [ ]:
class CrossAttention(nn.Module):
    def __init__(self, embed_dim, num_heads = 1, dropout = 0.0):
        super(CrossAttention, self).__init__()
        self.emb_dim = emb_dim
        self.num_heads = num_heads

        self.w_k = nn.Linear(emb_dim, emb_dim * num_heads, bias=False)
        self.w_q = nn.Linear(emb_dim, emb_dim * num_heads, bias=False)
        self.w_v = nn.Linear(emb_dim, emb_dim * num_heads, bias=False)
        self.w_out = nn.Linear(emb_dim * num_heads, emb_dim)

    def forward(self, x):

        b, t, _ = x.shape
        e = self.emb_dim
        h = self.num_heads
        keys = self.w_k(x).view(b, t, h, e)
        values = self.w_v(x).view(b, t, h, e)
        queries = self.w_q(x).view(b, t, h, e)

        keys = keys.transpose(2, 1)
        queries = queries.transpose(2, 1)
        values = values.transpose(2, 1)

        dot = queries @ keys.transpose(3, 2)
        dot = dot / np.sqrt(e)
        dot = F.softmax(dot, dim=3)

        out = dot @ values
        out = out.transpose(1,2).contiguous().view(b, t, h * e)
        out = self.w_out(out)
        return out

# Model

In [ ]:
from tqdm import tqdm 
from collections import defaultdict
from torch.utils.data import DataLoader, Dataset

import torch.optim as optim

from ray import tune

from sklearn.metrics import classification_report

## Initialize the model and training parameters

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_crossattn = CrossAttention(50)
model_crossattn = model_crossattn.to(device)

epochs = 4

optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

criterion = nn.CrossEntropyLoss()

## Hyper-parameter Tuning with Ray-Tune

In [ ]:
for i in range(10):
    train(modemodel_crossattnl, optimizer, train_loader)
    acc = test(model_crossattn, valid_loader)
    tune.report(mean_accuracy=acc)

analysis = tune.run(train_loader, config={"lr": tune.grid_search([0.001, 0.01, 0.1])})

print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))

# Get a dataframe for analyzing trial results.
df = analysis.dataframe()

## Train the model

In [ ]:
model_crossattn.train()

for epoch in range(epochs):
    print("Epoch ", epoch)

    with tqdm(train_loader, unit="batch") as tepoch:
        for inputs, labels, lengths in tepoch:
            tepoch.set_description(f"Epoch {epoch}")

            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            labels = torch.argmax(labels[:,1:], dim= 1)
            inputs = torch.transpose(inputs, 0, 1).unsqueeze(1)
            inputs = inputs.reshape(inputs.shape[0], 1, -1)

            print(inputs.shape)
            print(labels.shape)


            outputs = model_crossattn(inputs)

            print(outputs[0].shape, outputs[1].shape)
            
            logits = outputs[0].squeeze(0)
            print(logits.shape)

            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            #input()

## Evaluate the model

In [ ]:
model_crossattn.eval()

for epoch in range(epochs):
    print("Epoch ", epoch)

    with tqdm(test_loader, unit="batch") as tepoch:
        for inputs, labels, lengths in tepoch:
            tepoch.set_description(f"Epoch {epoch}")

            with torch.no_grad():
                inputs, labels = inputs.to(device), labels.to(device)

                labels = torch.argmax(labels[:,1:], dim= 1)
                inputs = torch.transpose(inputs, 0, 1).unsqueeze(1)
                inputs = inputs.reshape(inputs.shape[0], 1, -1)

                print(inputs.shape)
                print(labels.shape)

                outputs = model_crossattn(inputs)

                print(outputs[0].shape, outputs[1].shape)
                
                logits = outputs[0].squeeze(0)
                print(logits.shape)

                loss = criterion(logits, labels)

                sm = nn.Softmax(1)
                y_pred = torch.argmax(sm(logits), 1)
                print(y_pred.shape)

                print(classification_report(labels, y_pred))

                #input()

# If we just want to use the fused embeddings to directly classify

In [ ]:
logits = model.forward(outputs)

sm = nn.Softmax(1)
pred = sm(logits)

labels = {0: "Happy", 1: "Sad", 2: "Anger", 3: "Surprise", 4: "Disgust", 5: "Fear"}
class_label = torch.argmax(pred, 1)

print("Predicted Label:", class_label, "Corresponding Class:", labels[class_label])